Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px

import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

Reading the dataset with encoding parameter set to ‘latin1’

In [2]:
df = pd.read_csv('Corona_NLP - Corona_NLP.csv', encoding = 'latin-1')
df.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41157 entries, 0 to 41156
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   UserName       41157 non-null  int64 
 1   ScreenName     41157 non-null  int64 
 2   Location       32563 non-null  object
 3   TweetAt        41157 non-null  object
 4   OriginalTweet  41157 non-null  object
 5   Sentiment      41157 non-null  object
dtypes: int64(2), object(4)
memory usage: 1.9+ MB


In [4]:
df.shape

(41157, 6)

In [5]:
df.columns

Index(['UserName', 'ScreenName', 'Location', 'TweetAt', 'OriginalTweet',
       'Sentiment'],
      dtype='object')

In [6]:
df.isnull().sum()
#Only Location column contains 8590 null values. 
#But this column is not useful for our sentiment analysis.Hence we will neglect these null values.

UserName            0
ScreenName          0
Location         8594
TweetAt             0
OriginalTweet       0
Sentiment           0
dtype: int64

Removing handle null values .

In [7]:
df = df.fillna(method ='pad')
df.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,Vagabonds,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,Vagabonds,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [8]:
df['Sentiment'].value_counts()

Positive              11422
Negative               9917
Neutral                7713
Extremely Positive     6624
Extremely Negative     5481
Name: Sentiment, dtype: int64

Function for removing @user

In [9]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i,'',input_txt)
    return input_txt

Creating new column with removed @user

In [10]:
df['Tweet'] = np.vectorize(remove_pattern)(df['OriginalTweet'], '@[\w]*')
df.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment,Tweet
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral,https://t.co/iFz9FAn2Pa and https://t.co/xX...
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive,advice Talk to your neighbours family to excha...
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive,Coronavirus Australia: Woolworths to give elde...
3,3802,48754,Vagabonds,16-03-2020,My food stock is not the only one which is emp...,Positive,My food stock is not the only one which is emp...
4,3803,48755,Vagabonds,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative,"Me, ready to go at supermarket during the #COV..."


Removed HTTP And URLS from Tweet

In [11]:
df['Tweet'] = df['Tweet'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])
df.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment,Tweet
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral,
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive,advice Talk to your neighbours family to excha...
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive,Coronavirus Australia: Woolworths to give elde...
3,3802,48754,Vagabonds,16-03-2020,My food stock is not the only one which is emp...,Positive,My food stock is not the only one which is emp...
4,3803,48755,Vagabonds,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative,"Me, ready to go at supermarket during the #COV..."


Removed special characters, numbers, punctuations

In [12]:
df['Tweet'] = df['Tweet'].str.replace('[^a-zA-Z#]+',' ')
df.head()

C:\Users\ayush\AppData\Local\Temp\ipykernel_13684\2026002561.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Tweet'] = df['Tweet'].str.replace('[^a-zA-Z#]+',' ')


,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment,Tweet
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral,
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive,advice Talk to your neighbours family to excha...
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive,Coronavirus Australia Woolworths to give elder...
3,3802,48754,Vagabonds,16-03-2020,My food stock is not the only one which is emp...,Positive,My food stock is not the only one which is emp...
4,3803,48755,Vagabonds,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative,Me ready to go at supermarket during the #COVI...


Convert words to lower case

In [13]:
df["Tweet"] = df["Tweet"].str.lower()
df.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment,Tweet
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral,
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive,advice talk to your neighbours family to excha...
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive,coronavirus australia woolworths to give elder...
3,3802,48754,Vagabonds,16-03-2020,My food stock is not the only one which is emp...,Positive,my food stock is not the only one which is emp...
4,3803,48755,Vagabonds,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative,me ready to go at supermarket during the #covi...


In [14]:
import nltk 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ayush\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

Removing Stopwords

In [16]:
df['Tweet'].apply(lambda x: [item for item in x if item not in stop])
df.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment,Tweet
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral,
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive,advice talk to your neighbours family to excha...
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive,coronavirus australia woolworths to give elder...
3,3802,48754,Vagabonds,16-03-2020,My food stock is not the only one which is emp...,Positive,my food stock is not the only one which is emp...
4,3803,48755,Vagabonds,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative,me ready to go at supermarket during the #covi...


Creating new variable tokenized tweet 

In [17]:
tokenized_tweet = df['Tweet'].apply(lambda x: x.split())

In [18]:
from nltk.stem.porter import *
stemmer = PorterStemmer()

Applying stemmer for tokenized_tweet

In [19]:
tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x])
df.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment,Tweet
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral,
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive,advice talk to your neighbours family to excha...
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive,coronavirus australia woolworths to give elder...
3,3802,48754,Vagabonds,16-03-2020,My food stock is not the only one which is emp...,Positive,my food stock is not the only one which is emp...
4,3803,48755,Vagabonds,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative,me ready to go at supermarket during the #covi...


Joining tokens into one sentence

In [20]:
for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])

In [21]:
df['Tweet']  = tokenized_tweet
df.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment,Tweet
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral,
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive,advic talk to your neighbour famili to exchang...
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive,coronaviru australia woolworth to give elderli...
3,3802,48754,Vagabonds,16-03-2020,My food stock is not the only one which is emp...,Positive,my food stock is not the onli one which is emp...
4,3803,48755,Vagabonds,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative,me readi to go at supermarket dure the #covid ...


Converting the 'Extremely Positive' and 'Extremely Negative' Sentiments to 'Positive' and 'Negative' sentiments respectively

In [22]:
df['Sentiment'] = df['Sentiment'].replace(['Extremely Negative','Extremely Positive'],['Negative','Positive'])
df.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment,Tweet
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral,
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive,advic talk to your neighbour famili to exchang...
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive,coronaviru australia woolworth to give elderli...
3,3802,48754,Vagabonds,16-03-2020,My food stock is not the only one which is emp...,Positive,my food stock is not the onli one which is emp...
4,3803,48755,Vagabonds,16-03-2020,"Me, ready to go at supermarket during the #COV...",Negative,me readi to go at supermarket dure the #covid ...


Spitting Our Dataset into Training And Testing Dataset

In [23]:
from sklearn.model_selection import train_test_split

train,test = train_test_split(df,test_size = 0.2,random_state=0,stratify = df.Sentiment.values)
print("train shape : ", train.shape)
print("test shape : ", test.shape)

train shape :  (32925, 7)
test shape :  (8232, 7)


In [24]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
stop = list(stopwords.words('english'))
vectorizer = CountVectorizer(decode_error = 'replace',stop_words = stop)

Using Counter Vectorizer For Multi Class Classification

In [25]:
X_train = vectorizer.fit_transform(train.Tweet.values)
X_test = vectorizer.transform(test.Tweet.values)

Y_train = train.Sentiment.values
Y_test = test.Sentiment.values

print("X_train.shape : ", X_train.shape)
print("X_train.shape : ", X_test.shape)
print("y_train.shape : ", Y_train.shape)
print("y_valid.shape : ", Y_test.shape)

X_train.shape :  (32925, 27328)
X_train.shape :  (8232, 27328)
y_train.shape :  (32925,)
y_valid.shape :  (8232,)


Using Naive Bayes Classifier for MULTICLASS Classification

In [26]:
from sklearn.naive_bayes import MultinomialNB
naiveByes_clf = MultinomialNB()
naiveByes_clf.fit(X_train,Y_train)

MultinomialNB()

In [27]:
predict1=naiveByes_clf.predict(X_test)
print(np.concatenate((predict1.reshape(len(predict1),1), Y_test.reshape(len(Y_test),1)),1))

[['Positive' 'Positive']
 ['Neutral' 'Neutral']
 ['Positive' 'Positive']
 ...
 ['Positive' 'Negative']
 ['Positive' 'Positive']
 ['Positive' 'Positive']]


Confusion Matrix

In [28]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
cm = confusion_matrix(predict1,Y_test)
print(cm)
print(classification_report(predict1,Y_test))

[[2344  487  617]
 [  77  398   96]
 [ 659  658 2896]]
              precision    recall  f1-score   support

    Negative       0.76      0.68      0.72      3448
     Neutral       0.26      0.70      0.38       571
    Positive       0.80      0.69      0.74      4213

    accuracy                           0.68      8232
   macro avg       0.61      0.69      0.61      8232
weighted avg       0.75      0.68      0.71      8232



Accuracy Score

In [29]:
accuracy_score(Y_test, predict1)

0.68488824101069

Performing SVM Model

In [30]:
from sklearn.svm import SVC
classifier=SVC(kernel='linear',random_state=0)
classifier.fit(X_train,Y_train)

SVC(kernel='linear', random_state=0)

In [31]:
predict2=classifier.predict(X_test)
print(np.concatenate((predict2.reshape(len(predict2),1), Y_test.reshape(len(Y_test),1)),1))

[['Positive' 'Positive']
 ['Neutral' 'Neutral']
 ['Positive' 'Positive']
 ...
 ['Neutral' 'Negative']
 ['Positive' 'Positive']
 ['Positive' 'Positive']]


Confusion Matrix

In [32]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
cm = confusion_matrix(predict2,Y_test)
print(cm)
print(classification_report(predict2,Y_test))

[[2501  250  396]
 [ 229 1093  261]
 [ 350  200 2952]]
              precision    recall  f1-score   support

    Negative       0.81      0.79      0.80      3147
     Neutral       0.71      0.69      0.70      1583
    Positive       0.82      0.84      0.83      3502

    accuracy                           0.80      8232
   macro avg       0.78      0.78      0.78      8232
weighted avg       0.79      0.80      0.79      8232



Accuracy Score

In [33]:
accuracy_score(Y_test, predict2)

0.7951895043731778

Performing KNN Model

In [38]:
from sklearn.neighbors import KNeighborsClassifier
classifier2=KNeighborsClassifier(n_neighbors=3,metric='minkowski',p=2)
classifier2.fit(X_train,Y_train)



KNeighborsClassifier(n_neighbors=3)

In [39]:
predict3=classifier2.predict(X_test)
print(np.concatenate((predict3.reshape(len(predict3),1), Y_test.reshape(len(Y_test),1)),1))

[['Neutral' 'Positive']
 ['Neutral' 'Neutral']
 ['Neutral' 'Positive']
 ...
 ['Neutral' 'Negative']
 ['Neutral' 'Positive']
 ['Positive' 'Positive']]


Confusion Matrix

In [40]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
cm = confusion_matrix(predict3,Y_test)
print(cm)
print(classification_report(predict3,Y_test))

[[ 956  105  470]
 [1909 1354 2267]
 [ 215   84  872]]
              precision    recall  f1-score   support

    Negative       0.31      0.62      0.41      1531
     Neutral       0.88      0.24      0.38      5530
    Positive       0.24      0.74      0.36      1171

    accuracy                           0.39      8232
   macro avg       0.48      0.54      0.39      8232
weighted avg       0.68      0.39      0.39      8232



Accuracy Score

In [41]:
accuracy_score(Y_test, predict3)

0.38654033041788144